![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [86]:
import pandas as pd
import numpy as np

# Start coding here...
# Read the csv as a dataframe
df = pd.read_csv('bank_marketing.csv')
df.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,day,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,13,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,19,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,23,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,27,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,3,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   client_id       41188 non-null  int64  
 1   age             41188 non-null  int64  
 2   job             41188 non-null  object 
 3   marital         41188 non-null  object 
 4   education       41188 non-null  object 
 5   credit_default  41188 non-null  object 
 6   housing         41188 non-null  object 
 7   loan            41188 non-null  object 
 8   contact         41188 non-null  object 
 9   month           41188 non-null  object 
 10  day             41188 non-null  int64  
 11  duration        41188 non-null  int64  
 12  campaign        41188 non-null  int64  
 13  pdays           41188 non-null  int64  
 14  previous        41188 non-null  int64  
 15  poutcome        41188 non-null  object 
 16  emp_var_rate    41188 non-null  float64
 17  cons_price_idx  41188 non-null 

In [88]:
# Split the data
# client
client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']].copy()

# campaign - need to create: campaign_id, last_contact_date and need to rename
# columns
campaign = df[['client_id', 'campaign', 'duration', 'pdays', 'previous', 'poutcome', 'y']].copy()

# economics - need to rename some columns
economics = df[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']].copy()

In [89]:
# rename client df column
old_client_col = ['client_id']
new_client_col = ['id']
client.rename(columns=dict(zip(old_client_col, new_client_col)), inplace=True)

# rename campaign df column
old_campaign_col = ['duration', 'previous', 'y', 'poutcome', 'campaign']
new_campaign_col = ['contact_duration', 'previous_campaign_contacts', 'campaign_outcome', 'previous_outcome', 'number_contacts']
campaign.rename(columns=dict(zip(old_campaign_col, new_campaign_col)), inplace=True)

# rename economics df column
old_economics_col = ['euribor3m', 'nr_employed']
new_economics_col = ['euribor_three_months', 'number_employed']
economics.rename(columns=dict(zip(old_economics_col, new_economics_col)), inplace=True)

In [90]:
# clean education col in client
client['education'] = client['education'].str.replace('.', '_')
client['education'] = client['education'].replace('unknown', np.nan)

In [91]:
# clean periods from job column
client['job'] = client['job'].str.replace('.','')

# convert 'success' and 'failure' in campaign_outcome and previous_outcome
campaign['campaign_outcome'] = np.where(campaign['campaign_outcome']=='yes', 1, 0)
campaign['previous_outcome'] = np.where(campaign['previous_outcome']=='success', 1, np.where(campaign['previous_outcome']=='failure', 0, np.nan))

# add column campaign_id to campaign
campaign['campaign_id'] = 1

In [92]:
# add column last_contact_date yyyy-mm-yy where yyyy is 2022
campaign['last_contact_date'] = pd.to_datetime(str(2022)+'-'+df['month']+'-'+df['day'].astype('str'), format="%Y-%b-%d")

In [93]:
# saving dataframes as csv
client.to_csv('client.csv',index=False)
campaign.to_csv('campaign.csv',index=False)
economics.to_csv('economics.csv',index=False)

In [94]:
display(client.head())

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


In [95]:
display(campaign.head())

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,1,261,999,0,NaN,0,1,2022-05-13
1,1,1,149,999,0,NaN,0,1,2022-05-19
2,2,1,226,999,0,NaN,0,1,2022-05-23
3,3,1,151,999,0,NaN,0,1,2022-05-27
4,4,1,307,999,0,NaN,0,1,2022-05-03


In [96]:
display(economics.head())

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0


In [97]:
client_table = """
CREATE TABLE client(
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client FROM 'client.csv' DELIMITER ',' CSV HEADER
"""

campaign_table = """
CREATE TABLE campaign(
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL REFERENCES client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATETIME
);
\copy campaign FROM 'campaign.csv' DELIMITER ',' CSV HEADER
"""

economics_table = """
CREATE TABLE economics(
    client_id SERIAL REFERENCES client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics FROM 'economics.csv' DELIMITER ',' CSV HEADER
"""